In [8]:
!pip install pandas geopandas networkx shapely matplotlib


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\yimin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
# shortest_path_coords_json.py

import pandas as pd
import networkx as nx
from shapely import wkt
import json

# -------------------------------
# 1. Load weighted road edges
# -------------------------------
edges_path = "newbrunswick_edges_weighted.csv"

df = pd.read_csv(edges_path)
df["geometry"] = df["geometry"].apply(wkt.loads)

# Extract start & end coordinates for graph edges
df["start"] = df["geometry"].apply(lambda x: (x.coords[0][0], x.coords[0][1]))
df["end"]   = df["geometry"].apply(lambda x: (x.coords[-1][0], x.coords[-1][1]))

# -------------------------------
# 2. Build weighted graph
# -------------------------------
G = nx.Graph()
for _, row in df.iterrows():
    u, v = row["start"], row["end"]
    G.add_edge(u, v, weight=row["length"])

print(f"✅ Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

# -------------------------------
# 3. Helper: nearest node
# -------------------------------
def nearest_node(G, point):
    return min(G.nodes, key=lambda n: (n[0] - point[0]) ** 2 + (n[1] - point[1]) ** 2)

# Example start/end coordinates (longitude, latitude)
start_point = (-74.4517, 40.4898)  # near Joyce Kilmer Ave
end_point   = (-74.4442, 40.4968)  # near Albany Street

start_node = nearest_node(G, start_point)
end_node   = nearest_node(G, end_point)

print(f"\nStart node: {start_node}")
print(f"End node:   {end_node}")

# -------------------------------
# 4. Compute shortest path
# -------------------------------
try:
    path_nodes = nx.shortest_path(G, source=start_node, target=end_node, weight="weight")
    path_length = nx.shortest_path_length(G, source=start_node, target=end_node, weight="weight")

    print(f"\n✅ Shortest path found! Weighted distance: {path_length:.2f} meters")
    print(f"Total nodes in path: {len(path_nodes)}")

    # Convert to list of dicts or tuples
    coords_list = [{"lon": lon, "lat": lat} for lon, lat in path_nodes]

    # -------------------------------
    # 5. Save to JSON
    # -------------------------------
    json_out = "shortest_path.json"
    with open(json_out, "w") as f:
        json.dump(coords_list, f, indent=4)

    print(f"📄 Saved {len(coords_list)} coordinates to: {json_out}")

except nx.NetworkXNoPath:
    print("❌ No path found between the given points.")


✅ Graph built with 793 nodes and 1174 edges

Start node: (-74.4511747, 40.4894213)
End node:   (-74.4443072, 40.4967595)

✅ Shortest path found! Weighted distance: 1208.99 meters
Total nodes in path: 13
📄 Saved 13 coordinates to: shortest_path.json
